# **GOAL** : define the right structure of our data .

# **I-GOOGLE ACCOUNT AUTHENTIFICATION**

In [ ]:
from google.colab import auth
auth.authenticate_user()


# **II-IMPORT THE REQUIREMENTS LIBRARIES**

In [ ]:
# !pip install cupy
# import cupy as cp

In [ ]:
! pip install -q -U zarr xarray gcsfs
import xarray as xr
import gcsfs
import zarr
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd



In [ ]:

# define the neuralgcm path data
ngcm_path = 'gs://neuralgcm/amip_runs/v1_precip_stochastic_2_8_deg/2001-to-2021_128x64_gauss_37-level_stride3h.zarr'
#load the data
# ngcm_data = xr.open_zarr(ngcm_path,chunks={"time":100})
ngcm_data = xr.open_zarr(ngcm_path)

<ipython-input-4-cd22bc7a61c8>:5: FutureWarning: In a future version, xarray will not decode timedelta values based on the presence of a timedelta-like units attribute by default. Instead it will rely on the presence of a timedelta64 dtype attribute, which is now xarray's default way of encoding timedelta64 values. To continue decoding timedeltas based on the presence of a timedelta-like units attribute, users will need to explicitly opt-in by passing True or CFTimedeltaCoder(decode_via_units=True) to decode_timedelta. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  ngcm_data = xr.open_zarr(ngcm_path)


In [ ]:
#the size of the neural gcm data
ngcm_data.nbytes/1e9

1628.740724872

In [ ]:
print(list(ngcm_data.data_vars))

['evaporation', 'geopotential', 'precipitation_cumulative_mean', 'specific_cloud_ice_water_content', 'specific_cloud_liquid_water_content', 'specific_humidity', 'temperature', 'u_component_of_wind', 'v_component_of_wind']


# **Data Understanding**

In [ ]:
ngcm_data.info()

xarray.Dataset {
dimensions:
	time = 37 ;
	prediction_timedelta = 58408 ;
	surface = 1 ;
	longitude = 128 ;
	latitude = 64 ;
	level = 3 ;

variables:
	float32 evaporation(time, prediction_timedelta, surface, longitude, latitude) ;
	float32 geopotential(time, prediction_timedelta, level, longitude, latitude) ;
	float64 latitude(latitude) ;
	int64 level(level) ;
	float64 longitude(longitude) ;
	float32 precipitation_cumulative_mean(time, prediction_timedelta, surface, longitude, latitude) ;
	timedelta64[ns] prediction_timedelta(prediction_timedelta) ;
	float32 specific_cloud_ice_water_content(time, prediction_timedelta, level, longitude, latitude) ;
	float32 specific_cloud_liquid_water_content(time, prediction_timedelta, level, longitude, latitude) ;
	float32 specific_humidity(time, prediction_timedelta, level, longitude, latitude) ;
	int64 surface(surface) ;
	float32 temperature(time, prediction_timedelta, level, longitude, latitude) ;
	datetime64[ns] time(time) ;
	float32 u_component_o

In [ ]:
ngcm_data.precipitation_cumulative_mean


<xarray.DataArray 'precipitation_cumulative_mean' (time: 37,
                                                   prediction_timedelta: 58408,
                                                   surface: 1, longitude: 128,
                                                   latitude: 64)> Size: 71GB
dask.array<open_dataset-precipitation_cumulative_mean, shape=(37, 58408, 1, 128, 64), dtype=float32, chunksize=(1, 8, 1, 128, 64), chunktype=numpy.ndarray>
Coordinates:
  * latitude              (latitude) float64 512B -87.86 -85.1 ... 85.1 87.86
  * longitude             (longitude) float64 1kB 0.0 2.812 ... 354.4 357.2
  * prediction_timedelta  (prediction_timedelta) timedelta64[ns] 467kB 00:00:...
  * surface               (surface) int64 8B 1
  * time                  (time) datetime64[ns] 296B 2001-01-01 ... 2001-12-27

##❗ **Questions:**



*   I had a question about what the base date is for adding the prediction_timedelta, and to resolve it, by checking the number of values in prediction_timedelta and the values it contains, I concluded that the base time is the minimum of the time coordinate.



In [ ]:
#the min and the max values of time in the global neural GCM
print(f"the min date is :",ngcm_data.time.min().values)
print(f"the max date is :",ngcm_data.time.max().values)

the min date is : 2001-01-01T00:00:00.000000000
the max date is : 2001-12-27T00:00:00.000000000


In [ ]:
#the prediction time
ngcm_data.prediction_timedelta.indexes

Indexes:
    prediction_timedelta  TimedeltaIndex([   '0 days 00:00:00',    '0 days 03:00:00',
                   '0 days 06:00:00',    '0 days 09:00:00',
                   '0 days 12:00:00',    '0 days 15:00:00',
                   '0 days 18:00:00',    '0 days 21:00:00',
                   '1 days 00:00:00',    '1 days 03:00:00',
                ...
                '7299 days 18:00:00', '7299 days 21:00:00',
                '7300 days 00:00:00', '7300 days 03:00:00',
                '7300 days 06:00:00', '7300 days 09:00:00',
                '7300 days 12:00:00', '7300 days 15:00:00',
                '7300 days 18:00:00', '7300 days 21:00:00'],
               dtype='timedelta64[ns]', name='prediction_timedelta', length=58408, freq=None)

In [ ]:
#Calculate the exact range of days covered by NeuralGCM.
import pandas as pd

base_date = pd.Timestamp(ngcm_data.time.min().values)
offset = pd.Timedelta(days=7300, hours=21)
final_date = base_date + offset

print(final_date)


2020-12-27 21:00:00


In [ ]:
#Calculate the exact range of days covered by NeuralGCM.
import pandas as pd

base_date = pd.Timestamp(ngcm_data.time.max().values)
offset = pd.Timedelta(days=7300, hours=21)
final_date = base_date + offset

print(final_date)


2021-12-22 21:00:00



### 📊 **Dataset Variable Descriptions**


[source](https://github.com/neuralgcm/neuralgcm/blob/main/docs/data_preparation.ipynb)

NeuralGCM models take and produce data on defined on ERA5's 37 pressure levels, including the following variables, provided in SI units and on the NeuralGCM model's native grid:

    Inputs/outputs (on pressure levels): u_component_of_wind, v_component_of_wind, geopotential, temperature, specific_humidity, specific_cloud_ice_water_content, specific_cloud_liquid_water_content.
    Forcings (surface level only): sea_surface_temperature, sea_ice_cover

This dataset contains atmospheric and surface-level variables structured along multiple dimensions. Below is an explanation of each variable:

---

#### 📅 **Temporal Dimensions**

* **`time(time)`** *(datetime64\[ns])*:
  Represents the base timestamps of the forecasts. Each entry corresponds to a specific initialization time.

* **`prediction_timedelta(prediction_timedelta)`** *(timedelta64\[ns])*:
  Specifies the lead times (time offsets) from the base `time`.

---

#### 🌍 **Spatial Dimensions**

* **`longitude(longitude)`** *(float64)*:
  Array of longitudes in degrees (typically 0° to 360° or -180° to 180° depending on the convention used).

* **`latitude(latitude)`** *(float64)*:
  Array of latitudes in degrees (typically from -90° to 90°).

* **`level(level)`** *(int64)*:
  Pressure levels in hPa for variables defined in the atmosphere . This dimension is used for 3D atmospheric fields.

* **`surface(surface)`** *(int64)*:
  A scalar or categorical indicator to mark variables defined at the Earth's surface.

---

#### 🌦️ **Forecast Variables**

* **`evaporation(time, prediction_timedelta, surface, longitude, latitude)`** *(float32)*:
  Total evaporation at the surface (kg/m² or mm equivalent), forecasted over time.

* **`geopotential(time, prediction_timedelta, level, longitude, latitude)`** *(float32)*:
  Geopotential height at various pressure levels (m²/s²), often used to represent height contours in meteorology.

* **`precipitation_cumulative_mean(time, prediction_timedelta, surface, longitude, latitude)`** *(float32)*:
  Cumulative mean precipitation (total rainfall) at the surface.

* **`specific_cloud_ice_water_content(time, prediction_timedelta, level, longitude, latitude)`** *(float32)*:
  The concentration of ice water in clouds at different atmospheric levels.

* **`specific_cloud_liquid_water_content(time, prediction_timedelta, level, longitude, latitude)`** *(float32)*:
  The concentration of liquid water in clouds at different atmospheric levels.

* **`specific_humidity(time, prediction_timedelta, level, longitude, latitude)`** *(float32)*:
  The amount of water vapor per unit mass of air at given pressure levels.

* **`temperature(time, prediction_timedelta, level, longitude, latitude)`** *(float32)*:
  Air temperature at various atmospheric pressure levels (in Kelvin).

* **`u_component_of_wind(time, prediction_timedelta, level, longitude, latitude)`** *(float32)*:
  Zonal (east-west) component of the wind at different levels.

* **`v_component_of_wind(time, prediction_timedelta, level, longitude, latitude)`** *(float32)*:
  Meridional (north-south) component of the wind at different levels.

---
### ⚡**Analysis**

* we can conclude that the data of neural gcm is between **2001-01-01 00:00:00** and **2020-12-27 21:00:00**


| Variable Name                               | SI Unit             | Description                                                                                                                |
| ------------------------------------------- | ------------------- | -------------------------------------------------------------------------------------------------------------------------- |
| **evaporation**                             | **kg/m²** *(or mm)* | Total amount of water evaporated per unit area. In meteorology, **1 kg/m² = 1 mm** of water.                               |
| **geopotential**                            | **m²/s²**           | Gravitational potential energy per unit mass. Often divided by **g = 9.81 m/s²** to get **geopotential height in meters**. |
| **precipitation\_cumulative\_mean**         | **kg/m²** *(or mm)* | Cumulative mean surface precipitation. As with evaporation, **1 kg/m² = 1 mm** of rainfall.                                |
| **specific\_cloud\_ice\_water\_content**    | **kg/kg**           | Mass of ice water in clouds per unit mass of moist air.                                                                    |
| **specific\_cloud\_liquid\_water\_content** | **kg/kg**           | Mass of liquid water in clouds per unit mass of moist air.                                                                 |
| **specific\_humidity**                      | **kg/kg**           | Mass of water vapor per unit mass of moist air (dimensionless but expressed in kg/kg).                                     |
| **temperature**                             | **K** *(Kelvin)*    | Air temperature at different pressure levels.                                                                              |
| **u\_component\_of\_wind**                  | **m/s**             | Zonal (east-west) wind speed. Positive = eastward.                                                                         |
| **v\_component\_of\_wind**                  | **m/s**             | Meridional (north-south) wind speed. Positive = northward.                                                                 |


# **neuralGCM data for our domain**

In [ ]:
#select Horn Africa data
ngcm_data_africa = ngcm_data.sel(latitude=slice(-20,30),longitude=slice(15,60))


In [ ]:
ngcm_data_africa

<xarray.Dataset> Size: 57GB
Dimensions:                              (time: 37,
                                          prediction_timedelta: 58408,
                                          surface: 1, longitude: 16,
                                          latitude: 18, level: 3)
Coordinates:
  * latitude                             (latitude) float64 144B -18.14 ... 29.3
  * level                                (level) int64 24B 500 700 850
  * longitude                            (longitude) float64 128B 16.88 ... 5...
  * prediction_timedelta                 (prediction_timedelta) timedelta64[ns] 467kB ...
  * surface                              (surface) int64 8B 1
  * time                                 (time) datetime64[ns] 296B 2001-01-0...
Data variables:
    evaporation                          (time, prediction_timedelta, surface, longitude, latitude) float32 2GB dask.array<chunksize=(1, 8, 1, 16, 18), meta=np.ndarray>
    geopotential                         (time, prediction_timedelta, level, longitude, latitude) float32 7GB dask.array<chunksize=(1, 8, 3, 16, 18), meta=np.ndarray>
    precipitation_cumulative_mean        (time, prediction_timedelta, surface, longitude, latitude) float32 2GB dask.array<chunksize=(1, 8, 1, 16, 18), meta=np.ndarray>
    specific_cloud_ice_water_content     (time, prediction_timedelta, level, longitude, latitude) float32 7GB dask.array<chunksize=(1, 8, 3, 16, 18), meta=np.ndarray>
    specific_cloud_liquid_water_content  (time, prediction_timedelta, level, longitude, latitude) float32 7GB dask.array<chunksize=(1, 8, 3, 16, 18), meta=np.ndarray>
    specific_humidity                    (time, prediction_timedelta, level, longitude, latitude) float32 7GB dask.array<chunksize=(1, 8, 3, 16, 18), meta=np.ndarray>
    temperature                          (time, prediction_timedelta, level, longitude, latitude) float32 7GB dask.array<chunksize=(1, 8, 3, 16, 18), meta=np.ndarray>
    u_component_of_wind                  (time, prediction_timedelta, level, longitude, latitude) float32 7GB dask.array<chunksize=(1, 8, 3, 16, 18), meta=np.ndarray>
    v_component_of_wind                  (time, prediction_timedelta, level, longitude, latitude) float32 7GB dask.array<chunksize=(1, 8, 3, 16, 18), meta=np.ndarray>
Attributes:
    experiment_id:  121052169
    worker_id:      2

In [ ]:
#the size of the horn africa data in GO
ngcm_data_africa.nbytes/1e9

57.26086748

In [ ]:
#select the climate timeset where prediction_timedelta is a multiple of 6h

# Convert prediction_timedelta to a pandas Series for easier manipulation
prediction_timedelta_series = pd.Series(ngcm_data_africa.prediction_timedelta.values)

# Select values where prediction_timedelta is a multiple of 6 hours (in seconds)
selected_indices = prediction_timedelta_series[prediction_timedelta_series % pd.Timedelta(hours=6) == pd.Timedelta(0)].index

# Use the selected indices to filter the entire dataset
selected_data = ngcm_data_africa.isel(prediction_timedelta=selected_indices)

selected_data

<xarray.Dataset> Size: 29GB
Dimensions:                              (time: 37,
                                          prediction_timedelta: 29204,
                                          surface: 1, longitude: 16,
                                          latitude: 18, level: 3)
Coordinates:
  * latitude                             (latitude) float64 144B -18.14 ... 29.3
  * level                                (level) int64 24B 500 700 850
  * longitude                            (longitude) float64 128B 16.88 ... 5...
  * prediction_timedelta                 (prediction_timedelta) timedelta64[ns] 234kB ...
  * surface                              (surface) int64 8B 1
  * time                                 (time) datetime64[ns] 296B 2001-01-0...
Data variables:
    evaporation                          (time, prediction_timedelta, surface, longitude, latitude) float32 1GB dask.array<chunksize=(1, 8, 1, 16, 18), meta=np.ndarray>
    geopotential                         (time, prediction_timedelta, level, longitude, latitude) float32 4GB dask.array<chunksize=(1, 8, 3, 16, 18), meta=np.ndarray>
    precipitation_cumulative_mean        (time, prediction_timedelta, surface, longitude, latitude) float32 1GB dask.array<chunksize=(1, 8, 1, 16, 18), meta=np.ndarray>
    specific_cloud_ice_water_content     (time, prediction_timedelta, level, longitude, latitude) float32 4GB dask.array<chunksize=(1, 8, 3, 16, 18), meta=np.ndarray>
    specific_cloud_liquid_water_content  (time, prediction_timedelta, level, longitude, latitude) float32 4GB dask.array<chunksize=(1, 8, 3, 16, 18), meta=np.ndarray>
    specific_humidity                    (time, prediction_timedelta, level, longitude, latitude) float32 4GB dask.array<chunksize=(1, 8, 3, 16, 18), meta=np.ndarray>
    temperature                          (time, prediction_timedelta, level, longitude, latitude) float32 4GB dask.array<chunksize=(1, 8, 3, 16, 18), meta=np.ndarray>
    u_component_of_wind                  (time, prediction_timedelta, level, longitude, latitude) float32 4GB dask.array<chunksize=(1, 8, 3, 16, 18), meta=np.ndarray>
    v_component_of_wind                  (time, prediction_timedelta, level, longitude, latitude) float32 4GB dask.array<chunksize=(1, 8, 3, 16, 18), meta=np.ndarray>
Attributes:
    experiment_id:  121052169
    worker_id:      2

In [ ]:
selected_data.nbytes/1e9

28.63043404

In [ ]:
print(selected_data.info())

xarray.Dataset {
dimensions:
	time = 37 ;
	prediction_timedelta = 29204 ;
	surface = 1 ;
	longitude = 16 ;
	latitude = 18 ;
	level = 3 ;

variables:
	float32 evaporation(time, prediction_timedelta, surface, longitude, latitude) ;
	float32 geopotential(time, prediction_timedelta, level, longitude, latitude) ;
	float64 latitude(latitude) ;
	int64 level(level) ;
	float64 longitude(longitude) ;
	float32 precipitation_cumulative_mean(time, prediction_timedelta, surface, longitude, latitude) ;
	timedelta64[ns] prediction_timedelta(prediction_timedelta) ;
	float32 specific_cloud_ice_water_content(time, prediction_timedelta, level, longitude, latitude) ;
	float32 specific_cloud_liquid_water_content(time, prediction_timedelta, level, longitude, latitude) ;
	float32 specific_humidity(time, prediction_timedelta, level, longitude, latitude) ;
	int64 surface(surface) ;
	float32 temperature(time, prediction_timedelta, level, longitude, latitude) ;
	datetime64[ns] time(time) ;
	float32 u_component_of

In [ ]:
# Create the date dimension:
base_date = pd.Timestamp('2001-01-01')
dates = base_date + selected_data.prediction_timedelta.values
selected_data = selected_data.assign_coords(date=('prediction_timedelta', dates))
selected_data = selected_data.swap_dims({'prediction_timedelta': 'date'})

In [ ]:
selected_data

<xarray.Dataset> Size: 29GB
Dimensions:                              (time: 37, date: 29204, surface: 1,
                                          longitude: 16, latitude: 18, level: 3)
Coordinates:
  * latitude                             (latitude) float64 144B -18.14 ... 29.3
  * level                                (level) int64 24B 500 700 850
  * longitude                            (longitude) float64 128B 16.88 ... 5...
    prediction_timedelta                 (date) timedelta64[ns] 234kB 00:00:0...
  * surface                              (surface) int64 8B 1
  * time                                 (time) datetime64[ns] 296B 2001-01-0...
  * date                                 (date) datetime64[ns] 234kB 2001-01-...
Data variables:
    evaporation                          (time, date, surface, longitude, latitude) float32 1GB dask.array<chunksize=(1, 8, 1, 16, 18), meta=np.ndarray>
    geopotential                         (time, date, level, longitude, latitude) float32 4GB dask.array<chunksize=(1, 8, 3, 16, 18), meta=np.ndarray>
    precipitation_cumulative_mean        (time, date, surface, longitude, latitude) float32 1GB dask.array<chunksize=(1, 8, 1, 16, 18), meta=np.ndarray>
    specific_cloud_ice_water_content     (time, date, level, longitude, latitude) float32 4GB dask.array<chunksize=(1, 8, 3, 16, 18), meta=np.ndarray>
    specific_cloud_liquid_water_content  (time, date, level, longitude, latitude) float32 4GB dask.array<chunksize=(1, 8, 3, 16, 18), meta=np.ndarray>
    specific_humidity                    (time, date, level, longitude, latitude) float32 4GB dask.array<chunksize=(1, 8, 3, 16, 18), meta=np.ndarray>
    temperature                          (time, date, level, longitude, latitude) float32 4GB dask.array<chunksize=(1, 8, 3, 16, 18), meta=np.ndarray>
    u_component_of_wind                  (time, date, level, longitude, latitude) float32 4GB dask.array<chunksize=(1, 8, 3, 16, 18), meta=np.ndarray>
    v_component_of_wind                  (time, date, level, longitude, latitude) float32 4GB dask.array<chunksize=(1, 8, 3, 16, 18), meta=np.ndarray>
Attributes:
    experiment_id:  121052169
    worker_id:      2

#1-select the level pressure

1.1 level =500

In [ ]:
# Sélect one level and one surface(surface=1, level=0)
ds_subset_1 = selected_data.sel(level=selected_data.level[0],time=selected_data.time[-1])
ds_subset_1

<xarray.Dataset> Size: 303MB
Dimensions:                              (date: 29204, surface: 1,
                                          longitude: 16, latitude: 18)
Coordinates:
  * latitude                             (latitude) float64 144B -18.14 ... 29.3
    level                                int64 8B 500
  * longitude                            (longitude) float64 128B 16.88 ... 5...
    prediction_timedelta                 (date) timedelta64[ns] 234kB 00:00:0...
  * surface                              (surface) int64 8B 1
    time                                 datetime64[ns] 8B 2001-12-27
  * date                                 (date) datetime64[ns] 234kB 2001-01-...
Data variables:
    evaporation                          (date, surface, longitude, latitude) float32 34MB dask.array<chunksize=(8, 1, 16, 18), meta=np.ndarray>
    geopotential                         (date, longitude, latitude) float32 34MB dask.array<chunksize=(8, 16, 18), meta=np.ndarray>
    precipitation_cumulative_mean        (date, surface, longitude, latitude) float32 34MB dask.array<chunksize=(8, 1, 16, 18), meta=np.ndarray>
    specific_cloud_ice_water_content     (date, longitude, latitude) float32 34MB dask.array<chunksize=(8, 16, 18), meta=np.ndarray>
    specific_cloud_liquid_water_content  (date, longitude, latitude) float32 34MB dask.array<chunksize=(8, 16, 18), meta=np.ndarray>
    specific_humidity                    (date, longitude, latitude) float32 34MB dask.array<chunksize=(8, 16, 18), meta=np.ndarray>
    temperature                          (date, longitude, latitude) float32 34MB dask.array<chunksize=(8, 16, 18), meta=np.ndarray>
    u_component_of_wind                  (date, longitude, latitude) float32 34MB dask.array<chunksize=(8, 16, 18), meta=np.ndarray>
    v_component_of_wind                  (date, longitude, latitude) float32 34MB dask.array<chunksize=(8, 16, 18), meta=np.ndarray>
Attributes:
    experiment_id:  121052169
    worker_id:      2

1.2 level =700

In [ ]:
# Sélect one level and one surface(surface=1, level=0)
ds_subset_2 = selected_data.sel(level=selected_data.level[1],time=selected_data.time[-1])
ds_subset_2

<xarray.Dataset> Size: 303MB
Dimensions:                              (date: 29204, surface: 1,
                                          longitude: 16, latitude: 18)
Coordinates:
  * latitude                             (latitude) float64 144B -18.14 ... 29.3
    level                                int64 8B 700
  * longitude                            (longitude) float64 128B 16.88 ... 5...
    prediction_timedelta                 (date) timedelta64[ns] 234kB 00:00:0...
  * surface                              (surface) int64 8B 1
    time                                 datetime64[ns] 8B 2001-12-27
  * date                                 (date) datetime64[ns] 234kB 2001-01-...
Data variables:
    evaporation                          (date, surface, longitude, latitude) float32 34MB dask.array<chunksize=(8, 1, 16, 18), meta=np.ndarray>
    geopotential                         (date, longitude, latitude) float32 34MB dask.array<chunksize=(8, 16, 18), meta=np.ndarray>
    precipitation_cumulative_mean        (date, surface, longitude, latitude) float32 34MB dask.array<chunksize=(8, 1, 16, 18), meta=np.ndarray>
    specific_cloud_ice_water_content     (date, longitude, latitude) float32 34MB dask.array<chunksize=(8, 16, 18), meta=np.ndarray>
    specific_cloud_liquid_water_content  (date, longitude, latitude) float32 34MB dask.array<chunksize=(8, 16, 18), meta=np.ndarray>
    specific_humidity                    (date, longitude, latitude) float32 34MB dask.array<chunksize=(8, 16, 18), meta=np.ndarray>
    temperature                          (date, longitude, latitude) float32 34MB dask.array<chunksize=(8, 16, 18), meta=np.ndarray>
    u_component_of_wind                  (date, longitude, latitude) float32 34MB dask.array<chunksize=(8, 16, 18), meta=np.ndarray>
    v_component_of_wind                  (date, longitude, latitude) float32 34MB dask.array<chunksize=(8, 16, 18), meta=np.ndarray>
Attributes:
    experiment_id:  121052169
    worker_id:      2

1.3 level =850

In [ ]:
# Sélect one level and one surface(surface=1, level=0)
ds_subset_3 = selected_data.sel(level=selected_data.level[2],time=selected_data.time[-1])
ds_subset_3

<xarray.Dataset> Size: 303MB
Dimensions:                              (date: 29204, surface: 1,
                                          longitude: 16, latitude: 18)
Coordinates:
  * latitude                             (latitude) float64 144B -18.14 ... 29.3
    level                                int64 8B 850
  * longitude                            (longitude) float64 128B 16.88 ... 5...
    prediction_timedelta                 (date) timedelta64[ns] 234kB 00:00:0...
  * surface                              (surface) int64 8B 1
    time                                 datetime64[ns] 8B 2001-12-27
  * date                                 (date) datetime64[ns] 234kB 2001-01-...
Data variables:
    evaporation                          (date, surface, longitude, latitude) float32 34MB dask.array<chunksize=(8, 1, 16, 18), meta=np.ndarray>
    geopotential                         (date, longitude, latitude) float32 34MB dask.array<chunksize=(8, 16, 18), meta=np.ndarray>
    precipitation_cumulative_mean        (date, surface, longitude, latitude) float32 34MB dask.array<chunksize=(8, 1, 16, 18), meta=np.ndarray>
    specific_cloud_ice_water_content     (date, longitude, latitude) float32 34MB dask.array<chunksize=(8, 16, 18), meta=np.ndarray>
    specific_cloud_liquid_water_content  (date, longitude, latitude) float32 34MB dask.array<chunksize=(8, 16, 18), meta=np.ndarray>
    specific_humidity                    (date, longitude, latitude) float32 34MB dask.array<chunksize=(8, 16, 18), meta=np.ndarray>
    temperature                          (date, longitude, latitude) float32 34MB dask.array<chunksize=(8, 16, 18), meta=np.ndarray>
    u_component_of_wind                  (date, longitude, latitude) float32 34MB dask.array<chunksize=(8, 16, 18), meta=np.ndarray>
    v_component_of_wind                  (date, longitude, latitude) float32 34MB dask.array<chunksize=(8, 16, 18), meta=np.ndarray>
Attributes:
    experiment_id:  121052169
    worker_id:      2

In [ ]:
# import xarray as xr
# import pandas as pd

# # Step 1: Retrieve the dates from the first Dataset
# # Let's assume the time variable is called 'prediction_timedelta'
# base_date = pd.Timestamp('2001-01-01')  # or dates_ds.time.min().values if available

# # Create a true datetime series
# dates = base_date + selected_data.prediction_timedelta.values

# # Step 2: Add this 'date' dimension to small_data
# # Replace the 'prediction_timedelta' dimension with 'date'
# small_data = selected_data.assign_coords(date=('prediction_timedelta', dates))
# selected_data = small_data.swap_dims({'prediction_timedelta': 'date'})

# # # Step 3: Group by year

In [ ]:
# prompt: supprimer la cordinate time and rename cordinate date as time

# selected_data = selected_data.drop_vars('time')
# selected_data = selected_data.rename({'date': 'time'})

In [ ]:
#without pressure level
evaporation=selected_data.evaporation
precipitation_cumulative_mean=selected_data.precipitation_cumulative_mean


In [ ]:
evaporation

<xarray.DataArray 'evaporation' (time: 37, date: 29204, surface: 1,
                                 longitude: 16, latitude: 18)> Size: 1GB
dask.array<getitem, shape=(37, 29204, 1, 16, 18), dtype=float32, chunksize=(1, 8, 1, 16, 18), chunktype=numpy.ndarray>
Coordinates:
  * latitude              (latitude) float64 144B -18.14 -15.35 ... 26.51 29.3
  * longitude             (longitude) float64 128B 16.88 19.69 ... 56.25 59.06
    prediction_timedelta  (date) timedelta64[ns] 234kB 0 days 00:00:00 ... 73...
  * surface               (surface) int64 8B 1
  * time                  (time) datetime64[ns] 296B 2001-01-01 ... 2001-12-27
  * date                  (date) datetime64[ns] 234kB 2001-01-01 ... 2020-12-...

In [ ]:
#subset at pressure 500
specific_cloud_ice_water_content_500=ds_subset_1.specific_cloud_ice_water_content
u_component_of_wind_500=ds_subset_1.u_component_of_wind
v_component_of_wind_500=ds_subset_1.v_component_of_wind

In [ ]:
#subset at pressure 700
specific_cloud_ice_water_content_700=ds_subset_2.specific_cloud_ice_water_content
u_component_of_wind_700=ds_subset_2.u_component_of_wind
v_component_of_wind_700=ds_subset_2.v_component_of_wind

In [ ]:
#subset at pressure 850
specific_cloud_ice_water_content_850=ds_subset_3.specific_cloud_ice_water_content
u_component_of_wind_850=ds_subset_3.u_component_of_wind
v_component_of_wind_850=ds_subset_3.v_component_of_wind

In [ ]:
from google.cloud import storage

# Initialisation du client
storage_client = storage.Client()

# Sélection du bucket
bucket = storage_client.bucket('melvin_aims_bucket')

# # Sauvegarde d'un fichier
# blob = bucket.blob('chemin/destination/fichier.txt')
# blob.upload_from_filename('chemin/source/fichier.txt')

In [ ]:
# Récupérer les préfixes (dossiers)
blobs = bucket.list_blobs()
dossiers = set()

for blob in blobs:
    # Extraire le chemin du dossier
    parties = blob.name.split('/')
    if len(parties) > 1:
        dossiers.add(parties[0])

# Afficher les dossiers
print("Liste des dossiers :")
for dossier in sorted(dossiers):
    print(dossier)

Liste des dossiers :
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
FCSTNorm
NGCM
evaporation
tfrecords


# **Data Uploading in the bucket**

In [ ]:
from google.cloud import storage
import os
import pandas as pd

# Configuration du client Google Cloud Storage
storage_client = storage.Client()

# Spécifiez votre bucket
bucket_name = 'melvin_aims_bucket'
bucket = storage_client.bucket(bucket_name)

#**Variable of interest:**
* U_component_of_wind: minmax scaling
* V_component_of_wind : minmax scaling
* evaporation
* precipitation
* cloud_ice_water






## **1-evaporation**

## **2-precipitation_cumulative_mean**

## **3-specific_cloud_ice_water_content_500**

## **4-u_component_of_wind_500**

## **5-v_component_of_wind_500**

## **6-specific_cloud_ice_water_content_700**

## **7-u_component_of_wind_700**

## **8-v_component_of_wind_700**

## **9-specific_cloud_ice_water_content_850**

## **10-u_component_of_wind_850**

## **11-v_component_of_wind_850**

# **Data loading in the bucket**

In [ ]:
from google.cloud import storage
import xarray as xr

# Initialiser le client Google Cloud Storage
storage_client = storage.Client()

# Nom du bucket et chemin du fichier
bucket_name = 'melvin_aims_bucket'  # Remplacez par le nom de votre bucket
# The file path should be relative to the bucket root.
file_path = '2000/20000601_12.nc'  # Remplacez par le chemin de votre fichier .nc dans le bucket

# Créer une référence au fichier dans le bucket
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(file_path)

# Télécharger le fichier dans un fichier temporaire
temp_file = '/tmp/temp_file.nc'  # Vous pouvez changer le nom du fichier temporaire
blob.download_to_filename(temp_file)

# Ouvrir le fichier NetCDF avec xarray
ds = xr.open_dataset(temp_file)

# Afficher le contenu du dataset
print(ds)

# Supprimer le fichier temporaire (facultatif)
import os
os.remove(temp_file)

<xarray.Dataset> Size: 544kB
Dimensions:        (longitude: 352, latitude: 384, time: 1)
Coordinates:
  * longitude      (longitude) float32 1kB 19.15 19.25 19.35 ... 54.15 54.25
  * latitude       (latitude) float32 2kB -13.65 -13.55 -13.45 ... 24.55 24.65
  * time           (time) datetime64[ns] 8B 2000-06-01T12:00:00
Data variables:
    precipitation  (latitude, longitude) float32 541kB ...
Attributes:
    description:  6h average of IMERG V07 rainfall data in the ICPAC region


In [ ]:
ds

<xarray.Dataset> Size: 544kB
Dimensions:        (longitude: 352, latitude: 384, time: 1)
Coordinates:
  * longitude      (longitude) float32 1kB 19.15 19.25 19.35 ... 54.15 54.25
  * latitude       (latitude) float32 2kB -13.65 -13.55 -13.45 ... 24.55 24.65
  * time           (time) datetime64[ns] 8B 2000-06-01T12:00:00
Data variables:
    precipitation  (latitude, longitude) float32 541kB ...
Attributes:
    description:  6h average of IMERG V07 rainfall data in the ICPAC region

In [ ]:
# prompt: modifie le code precedent en sachant que voici la structure du bucket     melvin_aims_bucket/NGCM/{varname}/{var_name}/{year_str}/ngcm_{var_name}_2.8deg_6h_GHA_{date_str}h.nc'

from google.cloud import storage
import xarray as xr
import pandas as pd
import os
from tqdm.notebook import tqdm

# Configuration du client Google Cloud Storage
storage_client = storage.Client()

# Spécifiez votre bucket
bucket_name = 'melvin_aims_bucket'
bucket = storage_client.bucket(bucket_name)

# Liste des variables à sauvegarder
variables = {
    'u_component_of_wind_850': u_component_of_wind_850,
    }

for var_name, var_data in tqdm(variables.items()):
  # Itérer sur chaque date
  for date in var_data.date.values:
    # Formater la date pour le nom du fichier
    date_str = pd.to_datetime(date).strftime('%Y%m%d_%H')
    year_str = pd.to_datetime(date).strftime('%Y')

    # Créer le chemin du fichier  MODIFICATION ICI
    file_path = f'NGCM/{var_name}/{year_str}/{var_name}_{year_str}_ngcm_{var_name}_2.8deg_6h_GHA_{date_str}h.nc'

    # Créer un fichier temporaire
    temp_file_path = f'/tmp/{var_name}_{date_str}.nc'

    print(file_path)
    print(temp_file_path)

    # Sauvegarder le DataArray dans le fichier temporaire
    var_data.sel(date=date).to_netcdf(temp_file_path)

    # # Charger le fichier dans le bucket
    blob = bucket.blob(file_path)
    blob.upload_from_filename(temp_file_path)

    # Supprimer le fichier temporaire
    os.remove(temp_file_path)

    print(f"Fichier '{file_path}' chargé avec succès.")

  0%|          | 0/1 [00:00<?, ?it/s]

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
NGCM/u_component_of_wind_850/2015/u_component_of_wind_850_2015_ngcm_u_component_of_wind_850_2.8deg_6h_GHA_20150902_18h.nc
/tmp/u_component_of_wind_850_20150902_18.nc
Fichier 'NGCM/u_component_of_wind_850/2015/u_component_of_wind_850_2015_ngcm_u_component_of_wind_850_2.8deg_6h_GHA_20150902_18h.nc' chargé avec succès.
NGCM/u_component_of_wind_850/2015/u_component_of_wind_850_2015_ngcm_u_component_of_wind_850_2.8deg_6h_GHA_20150903_00h.nc
/tmp/u_component_of_wind_850_20150903_00.nc
Fichier 'NGCM/u_component_of_wind_850/2015/u_component_of_wind_850_2015_ngcm_u_component_of_wind_850_2.8deg_6h_GHA_20150903_00h.nc' chargé avec succès.
NGCM/u_component_of_wind_850/2015/u_component_of_wind_850_2015_ngcm_u_component_of_wind_850_2.8deg_6h_GHA_20150903_06h.nc
/tmp/u_component_of_wind_850_20150903_06.nc
Fichier 'NGCM/u_component_of_wind_850/2015/u_component_of_wind_850_2015_ngcm_u_component_of_wind_850_2.8deg_6h_GHA_20150